In [1]:
import sys
sys.path.append('..')

In [2]:
from matrix import *
from copy import deepcopy
import numpy as np

In [3]:
test_matrix = [
    [1, 3, 1],
    [1,1,4],
    [4,3,1]
]
matrix = [
    [5,8,-2],
    [7,-2,-4],
    [5,8,-1]
]

In [4]:
def sign(a):
    if a == 0:
        return 0
    return a/abs(a)

def calc_norm_row(matrix, row):
    sum_row = 0
    for i in range(row,len(matrix)):
        sum_row += matrix[i][row]**2
    sum_row = sum_row**0.5
    return sum_row

def calc_norm_row_d1(matrix, row):
    sum_row = 0
    for i in range(row+1,len(matrix)):
        sum_row += matrix[i][row]**2
    sum_row = sum_row**0.5
    return sum_row

def calc_norm_row_d2(matrix, row):
    sum_row = 0
    for i in range(row+2,len(matrix)):
        sum_row += matrix[i][row]**2
    sum_row = sum_row**0.5
    return sum_row

In [5]:
def calc_mult_val_matrix(val,matrix):
    matrix = deepcopy(matrix)
    for i in range(len(matrix)):
        for j in range(len(matrix[i])):
            matrix[i][j] *= val
    return matrix

In [6]:
def get_decomposition_qr(matrix):
    n  = len(matrix)
    Ak = deepcopy(matrix)
    Q = get_matrix_eye(n)
    E = get_matrix_eye(n)
    for k in range(n-1):
        V = []
        for i in range(n):
            vi = 0
            if i == k:
                a = Ak[i][k]
                vi = a + sign(a)*calc_norm_row(Ak,k)
            elif i > k:
                vi = Ak[i][k]
            V.append(vi)
        V = transporate([V])
        V_t = transporate(V)
        val_V_tV = calc_mult_matrix(V_t,V)[0][0]
        matrixVV_t = VV_t = calc_mult_matrix(V,V_t)

        H =  calc_diff_matrix(E, calc_mult_val_matrix(2/val_V_tV, matrixVV_t))
        Ak = calc_mult_matrix(H,Ak)
        Q = calc_mult_matrix(Q,H)
    return Q, Ak


In [7]:
Q, R = get_decomposition_qr(test_matrix)
np.around(calc_mult_matrix(R,Q),3)

array([[ 3.889, -3.75 ,  2.745],
       [-1.378, -0.121, -1.924],
       [ 3.355,  0.904, -0.767]])

In [8]:
def get_roots(A, i):
    sz = len(A)
    a11 = A[i][i]
    a12 = A[i][i + 1] if i + 1 < sz else 0
    a21 = A[i + 1][i] if i + 1 < sz else 0
    a22 = A[i + 1][i + 1] if i + 1 < sz else 0
    return np.roots((1, -a11 - a22, a11 * a22 - a12 * a21))

def finish_iter_for_complex(A, eps, i):
    Q, R = get_decomposition_qr(A)
    A_next = calc_mult_matrix(R,Q)
    lambda1 = get_roots(A, i)
    lambda2 = get_roots(A_next, i)
    return True if abs(lambda1[0] - lambda2[0]) <= eps and \
                abs(lambda1[1] - lambda2[1]) <= eps else False


In [9]:
def get_eigenvalue(A,eps,i):
    A_i = deepcopy(A)
    res = 0
    while True:
        Q, R = get_decomposition_qr(A_i)
        A_i = calc_mult_matrix(R,Q)
        if calc_norm_row_d1(A_i,i) <= eps:
         
            res = (A_i[i][i], False, A_i)
            break
        elif calc_norm_row_d2(A_i,i) <= eps and finish_iter_for_complex(A_i,eps,i):
         
            res = (get_roots(A_i, i), True, A_i)
            break
    return res

In [10]:
def QR_method(A, eps):
    res = []
    i = 0
    A_i = deepcopy(A)
    while i < len(A):
        eigenval = get_eigenvalue(A_i, eps, i)
        if eigenval[1]:
            res.extend(eigenval[0])
            i += 2
        else:
            res.append(eigenval[0])
            i += 1
        A_i = eigenval[2]
    return res

In [11]:
QR_method(test_matrix,0.01)


[6.343150205845844,
 (-1.6713158724790946+1.5521480525359899j),
 (-1.6713158724790946-1.5521480525359899j)]

In [12]:
QR_method(matrix,0.01)

[-5.352621408890763, 4.773255624149787, 2.5832273116495994]

In [13]:
np.linalg.eigvals(matrix)

array([-5.35092951,  4.75931693,  2.59161258])